In [1]:
from SimCsstLens.LensPop.Population import LensPopulation
from SimCsstLens import CosmologyDistance as CD
import math
import h5py
import numpy as np 
from multiprocessing import Pool
import os
from astropy.table import Table, vstack

this_cosmos = CD.CosmosDist(Om0=0.3, Ode0=0.7, h=0.7)
lens_pop = LensPopulation(
    vdisp_floor=100, 
    zl_max=2.0, 
    cosmo_dist=this_cosmos,
    src_catalog_type='lsst',
    bands=['g', 'r', 'i', 'z'],
)
sky_frac = 17500.0/41252.96
N_etgs = lens_pop.dfl_pop.number_of_etgs(sky_frac=sky_frac) #ideal lenses
Nsamples_per_draw = 500000
Ndraw = math.ceil(N_etgs/Nsamples_per_draw)

In [2]:
current_dir = os.getcwd()
def return_detected(count_draw, stack=False):
    table = Table()

    fn = h5py.File(f"./samples/lenses_{count_draw}.hdf5", "r")
    src_thetaE = fn['source/thetaE'][()]
    for ii in range(src_thetaE.shape[0]):
        table[f'thetaE_s{ii}'] =  src_thetaE[ii, :]
        table[f'mass_s{ii}'] = fn['source/einstein_mass'][()][ii, :]
        table[f'z_s{ii}'] = fn['source']['z'][()][ii, :]
        table[f're_s{ii}'] = fn['source']['Re'][()][ii, :]
        table[f'q_s{ii}'] = fn['source']['q'][()][ii, :]
        table[f'pa_s{ii}'] = fn['source']['pa'][()][ii, :]
        table[f'x_s{ii}'] = fn['source']['xs'][()][ii, :]
        table[f'y_s{ii}'] = fn['source']['ys'][()][ii, :]
        table[f'mag_g_s{ii}'] = fn['source']['app_mag_g'][()][ii, :]
        table[f'mag_r_s{ii}'] = fn['source']['app_mag_r'][()][ii, :]
        table[f'mag_i_s{ii}'] = fn['source']['app_mag_i'][()][ii, :]
        table[f'mag_z_s{ii}'] = fn['source']['app_mag_z'][()][ii, :]

    table['vdisp_l'] = fn['deflector/vdisp'][()]
    table['re_l'] = fn['deflector/Re'][()]
    table['q_l']= fn['deflector/q'][()]
    table['z_l'] = fn['deflector/z'][()]
    table['mag_g_l'] = fn['deflector/app_mag_g'][()]
    table['mag_r_l'] = fn['deflector/app_mag_r'][()]
    table['mag_i_l'] = fn['deflector/app_mag_i'][()]
    table['mag_z_l'] = fn['deflector/app_mag_z'][()]

    SNR = fn['Obs/SNR'][()] #shape: [nsrc, nband, n_ideal_lens]
    table['SNR_g'] = SNR[0,0,:]
    table['SNR_r'] = SNR[0,1,:]
    table['SNR_i'] = SNR[0,2,:]
    table['SNR_z'] = SNR[0,3,:]
    table['SNR_stack'] = SNR[0,-1,:]
    SNR_griz = SNR[0,0:-1,:]
    indices = np.argmax(SNR_griz, axis=0)
    table['best_band'] = np.array(['g', 'r', 'i', 'z'])[indices] 
    
    mu = fn['Obs/magnification'][()] #shape: [nsrc, nband, n_ideal_lens]
    table['mu'] = mu[0,-1,:]
    
    ring_cond = fn['Obs/ring_cond'][()] #shape: [nsrc, nband, n_ideal_lenses]
    table['if_ring'] = ring_cond[0,-1,:]

    obs_cond = fn['Obs/detect_cond'][()] #shape: [nsrc, nband, n_ideal_lenses, 3]
    
    fn.close()


    #stack band
    this_obs_cond = obs_cond.astype('int')
    this_obs_cond = this_obs_cond[0, -1, :, :] #src-0, band-stack, nlens, 3-condition
    this_obs_cond = np.sum(this_obs_cond, axis=1) #shape: [nlens]
    bool_cond = (this_obs_cond==3)
    table['if_obs_stack'] = bool_cond
        
    #individual band
    this_obs_cond = obs_cond.astype('int')
    this_obs_cond = this_obs_cond[:, :-1, :, :] #remove stack band
    cond = np.sum(this_obs_cond, axis=3)[0,:,:] #cond: [nband, nlens]
    bool_cond = (cond==3).astype('int') #cond: [nband, nlens]
    bool_cond = (np.sum(bool_cond, axis=0)>0) #bool_cond" [nlens]
    table['if_obs_single'] = bool_cond
    
    n_detect_lens = len(np.where(bool_cond)[0])
    n_ideal_lens = src_thetaE.shape[1]
    
    table['noise_seed'] = np.arange(n_ideal_lens, dtype='int')
    
    return n_detect_lens, n_ideal_lens, table

n_detect_lens, n_ideal_lens, table = return_detected(0, stack=True)
table[0:5]

thetaE_s0,mass_s0,z_s0,re_s0,q_s0,pa_s0,x_s0,y_s0,mag_g_s0,mag_r_s0,mag_i_s0,mag_z_s0,vdisp_l,re_l,q_l,z_l,mag_g_l,mag_r_l,mag_i_l,mag_z_l,SNR_g,SNR_r,SNR_i,SNR_z,SNR_stack,best_band,mu,if_ring,if_obs_stack,if_obs_single,noise_seed
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,bool,bool,bool,int64
1.3196268626551055,264596483383.2295,4.4275,0.6214770781714286,0.7320195457264107,30.232606079738332,-0.9861918644280009,0.7417372980109783,26.9092,26.7538,25.9745,25.8792,236.61722483491087,0.5571690004369176,0.6259450622792214,0.3459937760977172,20.782409392808418,18.83204064877745,18.17475897845127,17.80022039548253,0.7048565112305543,0.631598379841616,1.0664061300899874,0.7688413613416288,1.5908905321477644,i,2.2542780269433975,False,False,False,0
0.2628982421522554,107697898772.67445,2.1026,0.6440636974627642,0.3751999224665038,149.00957552532762,-0.17606625344185742,-0.1032518801156661,28.3583,26.7204,26.2342,25.436,257.28858169322496,0.7470423815276427,0.6255463368829426,1.627394493875766,28.30483934462755,27.727295909446024,25.518880857965126,23.945024399843728,0.19739048722095442,0.7020838108576656,0.9099398715425865,1.2471517501466791,1.5205370331919836,z,1.6314398542350301,False,False,False,1
0.455528638317329,89802659712.4682,1.992,0.21405690701800398,0.6790676760823076,161.2187200496799,-0.026692001077661447,-0.13686549385421765,26.8525,26.953,26.9133,26.7556,188.433096335156,0.33955591626005277,0.5914140293555509,0.8300650060508795,26.60896960261688,23.979865500498974,22.682484721081423,21.584296345264853,4.086974238673896,2.9150887680312945,2.447942645337709,1.7920101650427436,5.592760347432647,g,4.957530690182853,True,False,False,2
0.5187100015104474,104853333839.93933,2.4086,0.3549103011141491,0.3774124022100549,97.0243487738108,0.18558001987468914,-0.10227107948093166,27.2898,27.1836,27.3103,27.1178,190.56814153004288,0.28081382359970547,0.553840951913132,0.8366023799363632,26.47857501501799,23.82017670475901,22.53204488045073,21.411309924731015,1.3287211625531155,1.143783208190991,0.8294180310608903,0.6337493313468742,1.9620026914882625,g,3.4161742803139465,True,False,False,3
0.4735196019521154,11348065107.802736,1.2158,0.4018505776518614,0.6253148962902952,99.31970108438986,-0.013814757846944914,0.09304000465780164,27.963,27.7227,27.4532,26.858,135.5462238550863,1.6150597519666754,0.5994424873797054,0.09642910721983886,18.582058221299157,17.521907713544078,17.141470309819116,16.832414567307648,0.479397284558981,0.37189040919821065,0.3718538807640398,0.423323464464561,0.8045644385748075,g,3.796914873932796,False,False,False,4


In [3]:
pool = Pool(processes=64)
vars = list(zip(list(range(Ndraw)), [True]*Ndraw))
results = pool.starmap(return_detected, vars)

n_detect_lens = [item[0] for item in results]
n_ideal_lens = [item[1] for item in results]
n_detect_lens = sum(n_detect_lens)
n_ideal_lens = sum(n_ideal_lens)

tables = [item[2] for item in results]
stacked_table = vstack(tables)
stacked_table.write('lens_samples.fits', overwrite=True)

In [4]:
stacked_table[142114]

thetaE_s0,mass_s0,z_s0,re_s0,q_s0,pa_s0,x_s0,y_s0,mag_g_s0,mag_r_s0,mag_i_s0,mag_z_s0,vdisp_l,re_l,q_l,z_l,mag_g_l,mag_r_l,mag_i_l,mag_z_l,SNR_g,SNR_r,SNR_i,SNR_z,SNR_stack,best_band,mu,if_ring,if_obs_stack,if_obs_single,noise_seed
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,bool,bool,bool,int64
1.5168830187419187,268854314487.72906,1.8098,0.9635260849723911,0.2344240855115115,14.744980139832624,-1.187607790172714,-0.09078073060996318,22.9067,22.966,23.0278,22.9565,255.3788709962412,1.65428775000688,0.7294196940551096,0.23385006462450564,18.89954359464662,17.321305003637082,16.757649438636843,16.473135644273807,23.705179535384605,15.331664875479314,11.298925012078316,8.03632949726326,28.18526712460036,g,2.997196691730436,True,False,True,2172


In [5]:
len(stacked_table)

5051213

In [8]:
mask = stacked_table['if_obs_stack'].data
table_detect = stacked_table[mask]
table_detect.write("catalog_csv/csst_wf_stack.csv")
len(table_detect)

99679

In [9]:
mask = stacked_table['if_obs_single'].data
table_detect = stacked_table[mask]
table_detect.write("catalog_csv/csst_wf_single.csv")
len(table_detect)

49375